# Imports

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install wget
%pip install -q google-generativeai==0.3.1

# SQL Lite database download

In [12]:
file_name = '/content/drive/MyDrive/sales_db_sample/sales_dataset.db'

# Database schema

In [4]:
# DATABASE SCHEMA:
# TABLE sales (
#   order_id VARCHAR(255) PRIMARY KEY, -- Unique identifier for each order
#   date DATE,                         -- Date when the order was placed (format: DD:MM:YYYY)

#   -- Categorical columns
#   status VARCHAR(50),                -- Current status of the order
#                                      -- (e.g., 'shipped', 'shipped - delivered to buyer', 'cancelled', 'shipped - returned to seller',
#                                      -- 'shipped - picked up', 'pending', 'pending - waiting for pick up', 'shipped - returning to seller',
#                                      -- 'shipped - out for delivery', 'shipped - rejected by buyer', 'shipping', etc.)
#   sales_channel VARCHAR(50),         -- Channel through which the order was placed
#                                      -- (e.g., 'amazon.in', 'non-amazon')
#   category VARCHAR(50),              -- Type of product ordered
#                                      -- (e.g., 'set', 'kurta', 'western dress', 'top', 'ethnic dress', 'blouse', 'bottom', 'saree', 'dupatta', etc.)
#   size VARCHAR(10),                  -- Size of the product ordered
#                                      -- (e.g., 'm', 'l', 'xl', 'xxl', 's', '3xl', 'xs', '6xl', '5xl', '4xl', 'free', etc.)
#   courier_status VARCHAR(50),        -- Status of the courier handling the delivery
#                                      -- (e.g., 'shipped', 'unshipped', 'cancelled', etc.)
#   ship_city VARCHAR(100),            -- City where the order is shipped
#                                      -- (e.g., 'mumbai', 'thane', 'gurugram', ...., etc.)
#   ship_state VARCHAR(100),           -- State where the order is shipped
#                                      -- (e.g., 'maharashtra', 'haryana', 'kerala', .....,  etc.)
#   b2b TINYINT,                       -- Indicates whether the order is a B2B transaction
#                                      -- (0 = No, 1 = Yes)

#   -- Numerical columns
#   quantity INTEGER,                  -- Quantity of items ordered (ideal range: 1 to 13)
#   amount DECIMAL(10, 2)              -- Total amount for the order (calculated)
# );

# Gemini Model

In [13]:
import time
import sqlite3
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key = <Use API key here>)

generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

# Optional
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

model = genai.GenerativeModel(model_name = "gemini-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

def get_sql_query(prompt, input_text):
  prompt_parts = [prompt, input_text]
  response = model.generate_content(prompt_parts)
  return response.text

def retrieve_data_from_database(sql_query):
    connection = sqlite3.connect(file_name)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    connection.commit()
    connection.close()
    return rows

In [14]:
sql_query_synthesis_input_prompt = """
TASK:
You are an expert in converting English questions to SQL query!
Given a user query in natural language related to a sales database, generate the corresponding SQL query that retrieves the required information.

DATABASE INFORMATION:
The database contains details such as:
- Order details
- Order statuses
- Sales channels
- Product categories
- Shipping cities
- Transaction amounts

The user may use any product names, statuses, or locations in their query, and the system must map those user inputs to the correct columns.

QUERY REQUIREMENTS:
Ensure the SQL query can handle:
1. Complex requests:
   - Filtering by product category, status, date range, ship_city, or amount.
2. Aggregate functions:
   - Sum, average, count, etc.
3. Sorting:
   - Allow sorting of results based on columns.

The table includes both categorical and numerical columns.

SQL Query Format Example:
SELECT columns
FROM sales
WHERE conditions;

- Replace column names as appropriate for user inputs such as 'product', 'status', or 'city'.
- Ensure that ambiguous user terms (e.g., 'product name') are resolved based on the closest matching column.

DATABASE SCHEMA:
TABLE sales (
  order_id VARCHAR(255) PRIMARY KEY, -- Unique identifier for each order
  date DATE,                         -- Date when the order was placed (format: DD:MM:YYYY)

  -- Categorical columns
  status VARCHAR(50),                -- Current status of the order
                                     -- (e.g., 'shipped', 'shipped - delivered to buyer', 'cancelled', 'shipped - returned to seller',
                                     -- 'shipped - picked up', 'pending', 'pending - waiting for pick up', 'shipped - returning to seller',
                                     -- 'shipped - out for delivery', 'shipped - rejected by buyer', 'shipping', etc.)
  sales_channel VARCHAR(50),         -- Channel through which the order was placed
                                     -- (e.g., 'amazon.in', 'non-amazon')
  category VARCHAR(50),              -- Type of product ordered
                                     -- (e.g., 'set', 'kurta', 'western dress', 'top', 'ethnic dress', 'blouse', 'bottom', 'saree', 'dupatta', etc.)
  size VARCHAR(10),                  -- Size of the product ordered
                                     -- (e.g., 'm', 'l', 'xl', 'xxl', 's', '3xl', 'xs', '6xl', '5xl', '4xl', 'free', etc.)
  courier_status VARCHAR(50),        -- Status of the courier handling the delivery
                                     -- (e.g., 'shipped', 'unshipped', 'cancelled', etc.)
  ship_city VARCHAR(100),            -- City where the order is shipped
                                     -- (e.g., 'mumbai', 'thane', 'gurugram', ...., etc.)
  ship_state VARCHAR(100),           -- State where the order is shipped
                                     -- (e.g., 'maharashtra', 'haryana', 'kerala', .....,  etc.)
  b2b TINYINT,                       -- Indicates whether the order is a B2B transaction
                                     -- (0 = No, 1 = Yes)

  -- Numerical columns
  quantity INTEGER,                  -- Quantity of items ordered (ideal range: 1 to 13)
  amount DECIMAL(10, 2)              -- Total amount for the order (calculated)
);

EXAMPLES:

1. Simple Query
   Input_text:
   "Show me all orders that are shipped."
   SQL Query:
   SELECT *
   FROM sales
   WHERE status = 'shipped';

2. Complex Query
   Input_text:
   "Give me the total number of orders and the average amount for orders placed via amazon.in that were shipped to mumbai and shipped in august 2022."
   SQL Query:
   SELECT COUNT(order_id) AS total_orders, AVG(amount) AS average_amount
   FROM sales
   WHERE sales_channel = 'amazon.in'
     AND ship_city = 'mumbai'
     AND status = 'shipped'
     AND date BETWEEN '01-08-2022' AND '31-08-2022';

3. Query with Non-Exact Names
   Input_text:
   "How many products were delivered to Delhi and returned by the buyer?"
   SQL Query:
   SELECT COUNT(order_id) AS total_delivered_and_returned
   FROM sales
   WHERE ship_city = 'delhi'
     AND (status = 'shipped - delivered to buyer' OR status = 'shipped - returned to seller');

IMPORTANT:
Also the sql code should not have ``` in beginning or end and sql word in output
"""

correct_query_identification_system_prompt = """
TASK:
Your task is to evaluate SQL queries based on an input text.
You will choose the query that most accurately matches the intent of the input text, ensuring it retrieves the intended data from the nearest available columns.
Consider matching columns closely, even if an exact match isn't available, and return only the most accurate query.
"""

detailed_explanation_system_prompt = """
Task:
You will be provided with three pieces of information:
1. Input_text: A natural language query or request.
2. SQL_query: The corresponding SQL query used to fetch data from a database.
3. Response: The result returned by executing the SQL query.

Your task is to deeply analyze these inputs and generate Detailed Explanations:
- Use the DATABASE SCHEMA to provide a contextual and accurate breakdown of the data returned.
- Provide a detailed breakdown that includes a comprehensive interpretation of the natural language input, a deep analysis of the response data, and how the structure of the database schema informs this data.
- Highlight relationships between different columns in the schema, and how specific values returned align with the categories or data types defined in the schema.
- Your explanation should aim to uncover both explicit details and any hidden patterns or insights that enhance the user's understanding.

Additional Instructions:
- If the response contains a single value, clearly explain what that value represents based on the schema.
- If the response contains a list of tuples, analyze each tuple, explaining its components in line with their database definitions and what they represent.
- For numerical values (e.g., amounts, quantities), leverage schema details like ranges or categories to provide more context.
- For categorical data (e.g., product categories, statuses), explain how these categories relate to each other or reflect real-world use cases based on the schema.
- Focus on identifying patterns, trends, and hidden insights within the data, providing a more granular perspective that goes beyond surface-level interpretation.

DATABASE SCHEMA:
TABLE sales (
  order_id VARCHAR(255) PRIMARY KEY, -- Unique identifier for each order
  date DATE,                         -- Date when the order was placed (format: DD:MM:YYYY)

  -- Categorical columns
  status VARCHAR(50),                -- Current status of the order
                                     -- (e.g., 'shipped', 'shipped - delivered to buyer', 'cancelled', etc.)
  sales_channel VARCHAR(50),         -- Channel through which the order was placed
                                     -- (e.g., 'amazon.in', 'non-amazon')
  category VARCHAR(50),              -- Type of product ordered
                                     -- (e.g., 'set', 'kurta', 'western dress', etc.)
  size VARCHAR(10),                  -- Size of the product ordered
                                     -- (e.g., 'm', 'l', 'xl', etc.)
  courier_status VARCHAR(50),        -- Status of the courier handling the delivery
                                     -- (e.g., 'shipped', 'unshipped', 'cancelled', etc.)
  ship_city VARCHAR(100),            -- City where the order is shipped
                                     -- (e.g., 'mumbai', 'thane', etc.)
  ship_state VARCHAR(100),           -- State where the order is shipped
                                     -- (e.g., 'maharashtra', 'haryana', etc.)
  b2b TINYINT,                       -- Indicates whether the order is a B2B transaction
                                     -- (0 = No, 1 = Yes)

  -- Numerical columns
  quantity INTEGER,                  -- Quantity of items ordered (ideal range: 1 to 13)
  amount DECIMAL(10, 2)              -- Total amount for the order (calculated)
);

EXAMPLES:

1. Simple Query and Response:
   Input_text:
   "Give me the total number of orders and the average amount for orders placed via amazon.in that were shipped to Mumbai and shipped in August 2022."

   SQL_query:
   SELECT COUNT(order_id) AS total_orders, AVG(amount) AS average_amount
   FROM sales
   WHERE sales_channel = 'amazon.in'
    AND ship_city = 'mumbai'
    AND status = 'shipped'
    AND date BETWEEN '01-08-2022' AND '31-08-2022';

   Response:
   [(674, 659.00)]

   Detailed Explanation:
   - Input_text Interpretation: The user is requesting two specific metrics:
     1. The total number of orders placed through "amazon.in" and shipped to Mumbai in August 2022.
     2. The average amount spent on these orders.

   - Response Explanation: The response (674, 659.00) includes two key metrics:
     1. 674: The total number of orders meeting the specified criteria.
     2. 659.00: The average order value in INR (Indian Rupees), showing the typical amount customers spent on each order.

   - Schema-informed Insights:
     1. The column `sales_channel` filters results to 'amazon.in', and `ship_city` to 'Mumbai', aligning with categorical data in the schema.
     2. The date range query uses the `date` column in the format specified (DD:MM:YYYY).
     3. The numerical value for `amount` and its average calculation suggest mid-range pricing for products based on the schema description.

2. Complex Query and Response:
   Input_text:
   "Show me all orders that are shipped in Mumbai."

   SQL_query:
   SELECT *
   FROM sales
   WHERE status = 'shipped'
    AND ship_city = 'mumbai';

   Response:
   [('402-8764819-3937102', '10:04:2022', 'shipped', 'amazon.in', 'western dress', 'xl', 'shipped', 1, 744.0, 'mumbai', 'maharashtra', 0),
   ('402-8764819-3937102', '10:04:2022', 'shipped', 'amazon.in', 'western dress', 'xl', 'shipped', 1, 744.0, 'mumbai', 'maharashtra', 0)]

   Detailed Explanation:
   - Input_text Interpretation: The user is asking for all orders that have been shipped to Mumbai.

   - Response Explanation: The response is a list of tuples, each representing the complete information of a single order.

   - Schema-informed Insights:
     1. The `order_id` uniquely identifies each order, matching the schema.
     2. The `category` column shows the product category, 'western dress', as listed in the schema.
     3. The `size` is 'xl', which is one of the valid values from the schema's `size` column.
     4. The `amount` (744.0) reflects the calculated amount for the order, as indicated in the schema.
"""

def get_correct_query_input_prompt(input_text, df):
    queries_section = ""
    for i, query in enumerate(df["Query"], start=1):
        queries_section += f"Query {i}:\n{query}\n\n"

    sql_query_prompt = f"""Input Text:
"{input_text}"

SQL Queries:
{queries_section}Evaluate the queries based on the input text and the following criteria:
- Select the query that best retrieves the intended data from the nearest available columns in the dataframe.
- Match the columns to those referenced in the input text as closely as possible, even if an exact match isn't available.

Return only the most accurate query.

IMPORTANT:
Also the sql code should not have ``` in beginning or end and sql word in output"""

    return sql_query_prompt

def get_detailed_information_input_prompt(input_text, sql_query, response):
  system_prompt = f"""
  Input_text:
  {input_text}

  SQL_query:
  {sql_query}

  Response:
  {response}

  Provide a detailed breakdown that includes a comprehensive interpretation of the natural language input, a nuanced understanding of the SQL query, and a deep analysis of the response data. Your explanation should aim to uncover both explicit details and any hidden patterns or insights that enhance the user's understanding.
  - If the response contains a single value, clearly explain what that value represents.
  - If the response contains a list of tuples, analyze each tuple, explaining its components and what they represent.
  - For a list of tuples of numbers, focus on identifying patterns, trends, and any hidden insights within the numbers, providing a more granular perspective that goes beyond surface-level interpretation.
  """
  return system_prompt

def split_text_by_length(input_data, max_length=1000):
    input_str = str(input_data)
    chunks = [input_str[i:i + max_length] for i in range(0, len(input_str), max_length)][0]
    return chunks

In [15]:
def generate_and_execute_sql(prompt, input_text, max_tries=5):
    tries = 0
    error_messages = []
    input_texts = [input_text]
    successful_queries = []

    while tries < max_tries:
        print(f'TRIAL: {tries + 1}')
        try:
            start_time = time.time()
            # Assume get_sql_query is a pre-defined function to generate the SQL query
            generated_sql_query = get_sql_query(prompt, input_text)
            print('-' * 50)
            print(generated_sql_query)
            print('-' * 50)
            # Correct function call to retrieve data from the database
            records = retrieve_data_from_database(generated_sql_query)
            latency = time.time() - start_time

            # Storing successful query, response, and latency
            successful_queries.append({
                "query": generated_sql_query,
                "response": records,
                "latency": latency
            })
            print('SUCCEEDED')

            # Optimize for latency after at least one success
            if len(successful_queries) > 0:
                input_text = f"""Modify the last successful SQL query by making changes to it and optimizing it for latency.
                ENSURE that the NEW QUERY is DIFFERENT from the previous one while prioritizing faster execution.
                The last successful query was:
                {successful_queries[-1]["query"]}"""

        except Exception as e:
            print('FAILED')
            msg = str(e)
            error_messages.append(msg)

            # Evolve the prompt to correct the error in the next attempt
            input_text = f"""{input_text}
            Encountered an error: {msg}.
            To address this, please generate an alternative SQL query response that avoids this specific error.
            Follow the instructions mentioned above to remediate the error.

            Modify the below SQL query to resolve the issue and ensure it is not a repetition of the previously generated query:
            {generated_sql_query}

            Ensure the revised SQL query aligns precisely with the requirements outlined in the initial question.
            Additionally, please optimize the query for latency while maintaining correctness and efficiency."""

            input_texts.append(input_text)
            print('=' * 100)

        tries += 1  # Increment the tries counter at the end of each iteration

    # If no successful queries were made, return None
    if len(successful_queries) == 0:
        return None

    # Sort successful queries by latency and return as DataFrame
    successful_queries.sort(key=lambda x: x['latency'])
    df = pd.DataFrame([(q["query"], q["response"], q["latency"]) for q in successful_queries],
                      columns=["Query", "Result", "Latency"])
    return df

In [16]:
query = get_sql_query(sql_query_synthesis_input_prompt, "Show me all orders that are Shipped in Mumbai")
print("Query:")
print(query)
output = retrieve_data_from_database(query)
print("Retrieved Data:")
print(output)

Query:
SELECT *
FROM sales
WHERE status = 'shipped'
  AND ship_city = 'mumbai';
Retrieved Data:
[('402-8764819-3937102', '10:04:2022', 'shipped', 'amazon.in', 'western dress', 'xl', 'shipped', 1, 744.0, 'mumbai', 'maharashtra', 0), ('403-9088879-4789944', '27:05:2022', 'shipped', 'amazon.in', 'kurta', 's', 'shipped', 1, 549.0, 'mumbai', 'maharashtra', 0), ('408-1040742-1169120', '16:06:2022', 'shipped', 'amazon.in', 'kurta', 'xxl', 'shipped', 1, 380.0, 'mumbai', 'maharashtra', 0), ('171-3863669-4117946', '08:05:2022', 'shipped', 'amazon.in', 'kurta', '6xl', 'shipped', 1, 760.0, 'mumbai', 'maharashtra', 0), ('406-0132661-3034727', '18:04:2022', 'shipped', 'amazon.in', 'top', 'l', 'shipped', 1, 432.0, 'mumbai', 'maharashtra', 0), ('406-0214474-1670742', '28:04:2022', 'shipped', 'amazon.in', 'set', 'xxl', 'shipped', 1, 788.0, 'mumbai', 'maharashtra', 0), ('171-0075055-8629156', '14:05:2022', 'shipped', 'amazon.in', 'set', 'l', 'shipped', 1, 560.0, 'mumbai', 'maharashtra', 0), ('406-899385

In [ ]:
query = get_sql_query(sql_query_synthesis_input_prompt, "Give me the total number of orders and the average amount for orders placed via amazon.in that were shipped to mumbai and shipped in august 2022.")
print("Query:")
print(query)
output = retrieve_data_from_database(query)
print("Retrieve Data:")
print(output)

In [ ]:
result = generate_and_execute_sql(sql_query_synthesis_input_prompt, "How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022", max_tries=5)

TRIAL: 1
--------------------------------------------------
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022';
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022'
GROUP BY category, size, ship_state;
--------------------------------------------------
SUCCEEDED
TRIAL: 3
--------------------------------------------------
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta') AND size IN ('xl', 'l', 's') AND ship_state = 'maharashtra' AND date BETWEEN '01-08-2022' AND '31-08-2022'
GROUP BY category, size, ship_state
ORDER BY SUM(amount) DESC
LIMIT 1;
----------

In [ ]:
new_prompt = get_correct_query_input_prompt("How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022", result)
print("new_prompt:")
print(new_prompt)
generated_sql_query = get_sql_query(correct_query_identification_system_prompt, new_prompt)
print("generated_sql_query:")
print(generated_sql_query)
records = retrieve_data_from_database(generated_sql_query)
print("retrieve_records:")
print(records)

new_prompt:
Input Text:
"How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022"

SQL Queries:
Query 1:
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta') AND size IN ('xl', 'l', 's') AND ship_state = 'maharashtra' AND date BETWEEN '01-08-2022' AND '31-08-2022'
GROUP BY category, size, ship_state
ORDER BY SUM(amount) DESC
LIMIT 1;

Query 2:
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022'
GROUP BY category, size, ship_state;

Query 3:
SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022';

Query 4:
SELECT category, size, ship_state, SUM(amount) AS total_amount
FROM sales
WHERE category IN ('top', 'western dress', 'kurta') AND s

In [ ]:
response = split_text_by_length(records)
print("Splitted response:")
print(response)
detailed_explanation_prompt = get_detailed_information_input_prompt("How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022", generated_sql_query, response)
print("detailed_explanation_prompt:")
print(detailed_explanation_prompt)
detailed_explanation = get_sql_query(detailed_explanation_system_prompt, detailed_explanation_prompt)
print("Detailed explanation:")
print(detailed_explanation)

Splitted response:
[(82655.95,)]
detailed_explanation_prompt:

  Input_text:
  How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022

  SQL_query:
  SELECT SUM(amount)
FROM sales
WHERE category IN ('top', 'western dress', 'kurta') AND size IN ('xl', 'l', 's') AND ship_state = 'maharashtra' AND date BETWEEN '01-08-2022' AND '31-08-2022'
GROUP BY category, size, ship_state
ORDER BY SUM(amount) DESC
LIMIT 1;

  Response:
  [(82655.95,)]

  Provide a detailed breakdown that includes a comprehensive interpretation of the natural language input, a nuanced understanding of the SQL query, and a deep analysis of the response data. Your explanation should aim to uncover both explicit details and any hidden patterns or insights that enhance the user's understanding.
  - If the response contains a single value, clearly explain what that value represents.
  - If the response contains a list of tuples, analyze each tuple, explaining its components and 

In [ ]:
print(detailed_explanation)

**Input_text Interpretation:** The user is asking for the total sales of three specific product categories ('top', 'western dress', 'kurta') in three specific sizes ('xl', 'l', 's') in the state of Maharashtra in August 2022.

**SQL_query Breakdown:**
- The query first filters the `sales` table using the following criteria:
  - `category` IN ('top', 'western dress', 'kurta'): This condition ensures that only rows where the `category` column matches one of the three specified categories are included in the result.
  - `size` IN ('xl', 'l', 's'): This condition ensures that only rows where the `size` column matches one of the three specified sizes are included in the result.
  - `ship_state` = 'maharashtra': This condition ensures that only rows where the `ship_state` column matches 'maharashtra' are included in the result.
  - `date` BETWEEN '01-08-2022' AND '31-08-2022': This condition ensures that only rows where the `date` column falls within the specified date range are included in 

In [ ]:
def text_to_sql(input_text):
  result = generate_and_execute_sql(sql_query_synthesis_input_prompt, input_text, max_tries=1)
  query_correction_prompt = get_correct_query_input_prompt(input_text, result)
  print("query_correction_prompt:")
  print(query_correction_prompt)
  generated_sql_query = get_sql_query(correct_query_identification_system_prompt, query_correction_prompt)
  print("generated_sql_query:")
  print(generated_sql_query)
  records = retrieve_data_from_database(generated_sql_query)
  records = split_text_by_length(records)
  detailed_explanation_prompt = get_detailed_information_input_prompt(input_text, generated_sql_query, records)
  print("detailed_explanation_prompt:")
  print(detailed_explanation_prompt)
  detailed_explanation = get_sql_query(detailed_explanation_system_prompt, detailed_explanation_prompt)
  return detailed_explanation

In [ ]:
%%time

detailed_explanation = text_to_sql("How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022")
print("detailed_explanation:")
print(detailed_explanation)

TRIAL: 1
--------------------------------------------------
SELECT SUM(amount) AS total_sales
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022';
--------------------------------------------------
SUCCEEDED
query_correction_prompt:
Input Text:
"How much total sell of top, western dress and kurta of size xl, l and s in maharashtra in august 2022"

SQL Queries:
Query 1:
SELECT SUM(amount) AS total_sales
FROM sales
WHERE category IN ('top', 'western dress', 'kurta')
  AND size IN ('xl', 'l', 's')
  AND ship_state = 'maharashtra'
  AND date BETWEEN '01-08-2022' AND '31-08-2022';

Evaluate the queries based on the input text and the following criteria:
- Select the query that best retrieves the intended data from the nearest available columns in the dataframe.
- Match the columns to those referenced in the input text as closely as possible, even if an exact match isn'